In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
tr_path = '../data/processed/training-set.csv'
val_path = '../data/processed/validation-set.csv'
tt_path = '../data/processed/testing-set.csv'

In [3]:
class NUSW_NB15_Dataset(Dataset):
    def __init__(self, path, mode='classification'):
        ds_type = path.split('/')[-1].split('-')[0]
        df = pd.read_csv(path)
        
        if mode == 'classification':
            x = df.drop(['id', 'attack_cat', 'label'], axis=1)
            y = pd.Categorical(df['attack_cat']).codes.astype(np.float64)
        else:
            x = df.drop(['id', 'attack_cat', 'label'], axis=1)
            y = df['label'].astype(np.float64)

        self.x = torch.FloatTensor(x.to_numpy())
        self.y = torch.Tensor(y)

        self.dim = self.x.shape[1]

        print(
            f'Finished reading the {ds_type} set of Dataset '\
            f'({len(self.x)} samples found, each dim = {self.dim})'
        )

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

In [4]:
def prep_dataloader(path, mode, batch_size, shuffle):
    dataset = NUSW_NB15_Dataset(path, mode)
    dataloader = DataLoader(
        dataset,
        batch_size,
        shuffle
    )
    return dataloader

In [5]:
mode = 'classification'

tr_dl = prep_dataloader(
    tr_path,
    mode=mode,
    batch_size=64,
    shuffle=True
)

val_dl = prep_dataloader(
    val_path,
    mode=mode,
    batch_size=64,
    shuffle=True
)

tt_dl = prep_dataloader(
    tt_path,
    mode=mode,
    batch_size=64,
    shuffle=False
)

Finished reading the training set of Dataset (164910 samples found, each dim = 196)
Finished reading the validation set of Dataset (41228 samples found, each dim = 196)
Finished reading the testing set of Dataset (51535 samples found, each dim = 196)


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(196, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits